## DSP Assignment-2: Transform Analysis: Examine the example
# 學號:711481109  姓名:莊祐儒

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

ck =[0.95*np.exp(1j*0.17*np.pi),    #ck value vector
     0.95*np.exp(1j*0.19*np.pi),
     0.95*np.exp(1j*0.21*np.pi),
     0.95*np.exp(1j*0.23*np.pi),
     ]

 ## 1. 建立極點/零點參數 ck矩陣
- ck 裡存的是複數極（或零）位置。 r=0.95（在單位圓內，表示穩定）、角度為 0.17π、0.19π、0.21π、0.23π。

In [ ]:
#H1(Z)
a1 = np.polymul([1, -0.98*np.exp(1j*0.8*np.pi)], [1, -0.98*np.exp(-1j*0.8*np.pi)])
b1 = np.polymul([1, -0.8*np.exp(1j*0.4*np.pi)], [1, -0.8*np.exp(-1j*0.4*np.pi)])

##  2. H1(Z) — 第一段濾波器
- a1 為分子係數 ，b1 為分母係數 。
- 使用 `np.polymul` 將共軛複數根的多項式相乘，確保係數為實數。

In [ ]:
#H2(Z)
a2 = [1.0]
b2 = [1.0]


for c in ck:
    ak = [abs(c)**2, -2*np.real(c), 1]
    bk = [1, -2*np.real(c), abs(c)**2]
    ak = np.polymul(ak, ak)
    bk = np.polymul(bk, bk)
    a2 = np.polymul(a2, ak)
    b2 = np.polymul(b2, bk)


## 3. H2(Z) — 第二段濾波器 (多個 ck 組成)
- ak 和 bk 是每個零點和極點對應的二階多項式，`np.polymul(ak, ak)`表示將二階多項式平方，累積多段濾波器的效果。
- a2 與 b2 分別累乘所有段的分子和分母，最後得到 H2(Z) 的完整多項式。

In [ ]:
#H(Z) = H1(Z)*H2(Z)
a = np.polymul(a1, a2)
b = np.polymul(b1, b2)

w = np.linspace(-np.pi, np.pi, 1024)
z = np.exp(1j*w)
H = np.polyval(a[::-1], z**-1) / np.polyval(b[::-1], z**-1)

##  4. 串接H1(z)和H2(z)
- 將第一段濾波器 H1(Z) 與第二段濾波器 H2(Z) 相乘，得到整體系統 H(Z) 的分子 a 與分母 b。
- w 是從 -π 到 π 的離散頻率範圍 (rad/sample)。
$$
z = exp(j*w) 是對應 z 平面單位圓的點。
$$
- 在 Python 中，`np.polyval(p, x)` 的作用是計算多項式在指定點的值：

$$
p(x) = p_0 x^n + p_1 x^{n-1} + ... + p_n
$$

- 這裡 `p[0]` 對應最高次項，`p[-1]` 對應常數項（降冪順序）。  
- 若要對應 升冪順序 (從 x^0 到 x^n)，需要使用 `p[::-1]` 反轉係數順序。

在數位濾波器的 Z 轉換中，系統函數表示為：

$$
H(z) = \frac{a_0 + a_1 z^{-1} + \dots + a_M z^{-M}}{b_0 + b_1 z^{-1} + ... + b_N z^{-N}}
$$

- 這裡分子與分母是 **降冪形式**，即 z^0, z^-1, … z^-N。  
- 所以在 Python 計算時，要將多項式係數倒轉 (`a[::-1]` 或 `b[::-1]`) 並帶入 `z**-1`。



In [ ]:
# phase
phase_principal = np.angle(H)          # principal value in  [-π, π]
phase_unwrap = np.unwrap(phase_principal)  # unwrapped phase
phase_unwrap = phase_unwrap 

#principal value
plt.figure(figsize=(10, 4))
plt.plot(w, phase_principal, label='Principal Phase')
plt.title("Principal Value of Phase Response")
plt.xlabel(r'$\omega$ (rad/sample)')
plt.ylabel(r'ARG$|H(e^{j\omega})|$')
xticks = np.arange(-np.pi, np.pi+0.01, 0.2*np.pi)
xtick_labels = [f"{t/np.pi:.1f}π" if abs(t) > 1e-10 else "0" for t in xticks]
plt.xticks(xticks, xtick_labels)
plt.grid(True)
plt.show()

# continuous phase
plt.figure(figsize=(10, 4))
plt.plot(w, phase_unwrap, label='Unwrapped Phase')
plt.title("Unwrapped Phase Response")
plt.xlabel(r'$\omega$ (rad/sample)')
plt.ylabel(r'arg$|H(e^{j\omega})|$')
xticks = np.arange(-np.pi, np.pi+0.01, 0.2*np.pi)
xtick_labels = [f"{t/np.pi:.1f}π" if abs(t) > 1e-10 else "0" for t in xticks]
plt.xticks(xticks, xtick_labels)
plt.yticks(np.arange(-100, 25, 25))
plt.grid(True)
plt.show()

## 5.Phase (principal value and continuous phase)作圖
- `np.angle(H)` 回傳每個頻率點的複數相位（主值），範圍在 -π 到 π
- `np.unwrap` 將相位的 2π 跳變修正，使相位曲線連續，方便計算群延遲或進行相位補償。
- X 軸刻度用 0.2π 為單位顯示，使頻率更直覺。
- Y 軸刻度根據幅值範圍設定，方便觀察。
- `plt.grid(True)`：增加網格，便於對比幅值。

### 圖片展示

![Screen 1](https://raw.githubusercontent.com/Rickee12/DSP-Assignment-2-Transform-Analysis-Examine-the-example/main/figure2/screen1.png)


![Screen 2](https://raw.githubusercontent.com/Rickee12/DSP-Assignment-2-Transform-Analysis-Examine-the-example/main/figure2/screen2.png)






In [ ]:
# groupdelay τg = -dφ/dω
group_delay = -np.gradient(phase_unwrap, w)

#group delay
plt.plot(w, group_delay)
plt.title("Group Delay")
plt.xlabel(r'$\omega$ (rad/sample)')
plt.ylabel(r'grd$|H(e^{j\omega})|$')
xticks = np.arange(-np.pi, np.pi+0.01, 0.2*np.pi)
xtick_labels = [f"{t/np.pi:.1f}π" if abs(t) > 1e-10 else "0" for t in xticks]
plt.xticks(xticks, xtick_labels)
plt.grid(True)
plt.show()

## 6.Group delay
### 群延遲定義為：

$$
\tau_g(\omega) = -\frac{d\phi(\omega)}{d\omega}
$$

其中 $\phi(\omega)$ 為訊號的連續相位 (unwrapped phase)。

在程式中使用 `numpy.gradient` 計算數值微分

- X 軸刻度用0.2π 為單位顯示，使頻率更直覺。
- Y 軸刻度根據幅值範圍設定，方便觀察。

### 圖片展示
 
![Screen 3](https://raw.githubusercontent.com/Rickee12/DSP-Assignment-2-Transform-Analysis-Examine-the-example/main/figure2/screen3.png)


In [ ]:
#Amplitude and Frequency Response
H_mag = np.abs(H)
plt.figure(figsize=(10, 4))
plt.plot(w, H_mag, 'b', linewidth=1.2)
plt.title("Magnitude of frequency response")
plt.xlabel(r'$\omega$ (rad/sample)')
plt.ylabel(r'$|H(e^{j\omega})|$')
# X-axis 
xticks = np.arange(-np.pi, np.pi+0.01, 0.2*np.pi)
xtick_labels = [f"{t/np.pi:.1f}π" if abs(t) > 1e-10 else "0" for t in xticks]
plt.xticks(xticks, xtick_labels)
# Y-axis
plt.yticks(np.arange(0, 8, 1))
plt.grid(True)
plt.show()


## 7. Magnitude of frequency response
- `np.abs(H)`：取頻率響應的幅值。
- w：數位角頻率範圍 [-π, π]。
- X 軸刻度用0.2π 為單位顯示，使頻率更直覺。
- Y 軸刻度根據幅值範圍設定，方便觀察。
- `plt.grid(True)`：增加網格，便於對比幅值。

### 圖片展示
![Screen 4](https://raw.githubusercontent.com/Rickee12/DSP-Assignment-2-Transform-Analysis-Examine-the-example/main/figure2/screen4.png)


In [ ]:
# x[n]
N = 300  #sample number
hann_len = 60  #hanning window length
n = np.arange(N)   #Sample index of a discrete-time signal

# Three sinusoidal angular frequencies (rad/sample)
w1 = 0.8*np.pi   
w2 = 0.2*np.pi
w3 = 0.4*np.pi

hann_win = np.hanning(hann_len)   # Hanning window
start_idxs = [0, 60, 120]        # Starting indexs of the three windows

x = np.zeros(N)   # signal initialization

for w, s in zip([w1, w2, w3], start_idxs):
    n_local = np.arange(hann_len)               # index within the window
    sinusoid = np.sin(w * n_local)              # single-frequency sine within the window
    x[s:s+hann_len] += sinusoid * hann_win      # add to the total signal


## 8.x[n] (輸入信號)
- 使用 Hanning 窗來避免訊號突變造成頻譜泄漏。

- 將三個不同頻率的正弦波放置在不同起始位置，使訊號在時間上分段。

- `x` 最終為疊加後的加窗訊號。

In [ ]:
# time domain plot
plt.figure(figsize = (10, 3)) 
plt.plot(n, x, lw =1)
plt.xlim(0,N-1)
plt.xlabel('sample number (n)')
plt.ylabel('x[n]')
plt.title('Waveform of signal x[n]')
plt.grid(True)

#frequency domain plot
Nfft = 8192                   #Set the FFT length to 8192
X = np.fft.fft(x,Nfft)        #fft
X = np.fft.fftshift(X)        #shift zero freq to center
freqs = np.linspace(-np.pi, np.pi, Nfft, endpoint = False)
plt.figure(figsize = (10, 3)) 
plt.plot(freqs, np.abs(X))
plt.xlim(-np.pi, np.pi)
plt.xlabel(r'$\omega$ (rad/sample)')
plt.ylabel(r'$|X(e^{j\omega})|$')
plt.title('Magnitude of DTFT of x[n]')
xticks = np.arange(-np.pi, np.pi + 0.1, 0.2*np.pi)      # x 軸刻度每 0.2π
xtick_labels = [f'{t/np.pi:.1f}π' if t != 0 else '0' for t in xticks]  # 顯示成 π 單位
plt.xticks(xticks, xtick_labels)
plt.grid(True)
plt.show()


## 9. x[n] 的時域波形及頻域波形
- `x`用來代表時域，`X`用來代表頻域
- 時域波形: 顯示加窗後訊號的樣本值變化。

- 頻域波形: 使用 FFT 計算離散時間傅立葉變換 (DTFT) 的近似，並用 `fftshift` 將零頻置中，方便觀察正負頻率成分。

- x 軸以弧度/樣本 (rad/sample) 表示，並用 0.2π 為單位顯示。

### 圖片展示
![Screen 5](https://raw.githubusercontent.com/Rickee12/DSP-Assignment-2-Transform-Analysis-Examine-the-example/main/figure2/screen5.png)


![Screen 6](https://raw.githubusercontent.com/Rickee12/DSP-Assignment-2-Transform-Analysis-Examine-the-example/main/figure2/screen6.png)


In [ ]:
# 初始化輸出 y 為複數
y = np.zeros(N, dtype=complex)

# 差分方程計算
for n_idx in range(N):
    num_sum = sum(a[k] * x[n_idx - k] for k in range(len(a)) if n_idx - k >= 0)
    den_sum = sum(b[k] * y[n_idx - k] for k in range(1, len(b)) if n_idx - k >= 0)
    y[n_idx] = (num_sum - den_sum) / b[0]

# --- 時域圖 ---
plt.figure(figsize=(10,3))
plt.plot( y)   # 只畫實部
plt.xlabel('sample number (n)')
plt.ylabel('y[n]')
plt.title('Filtered Signal y[n] ')
plt.grid(True)
plt.show()

### 10. y[n] (輸出信號)


- `y` 為濾波後輸出，初始化為複數型態，避免運算丟失虛部。

- 差分方程公式：

$$
y[n] = \frac{\sum_{k=0}^{M-1} a[k]\, x[n-k] - \sum_{k=1}^{N-1} b[k]\, y[n-k]}{b[0]}
$$

  - 前饋項（分子）對應濾波器的零點 。  
  - 反饋項（分母）對應濾波器的極點 。  

- 最後繪製濾波後訊號的時域波形。

### 圖片展示
![Screen 7](https://raw.githubusercontent.com/Rickee12/DSP-Assignment-2-Transform-Analysis-Examine-the-example/main/figure2/screen7.png)
